# Линейная регрессия

В этой записной книжке рассматривается решение на основе Python для первого упражнения по программированию из курса машинного обучения на Coursera. Пожалуйста, обратитесь к [тексту упражнения] (https://github.com/jdwittenauer/ipython-notebooks/blob/master/exercises/ML/ex1.pdf) для получения подробных описаний и уравнений.

В этом упражнении мы реализуем простую линейную регрессию с использованием градиентного спуска и применим ее к модельной задачи. Мы также расширим нашу реализацию на случай нескольких переменных и применим ее к немного более сложному примеру.

## С одной переменной

### Знакомство с данными

В первой части упражнения перед нами стоит задача реализовать линейную регрессию с одной переменной для прогнозирования прибыли фургона с едой. Предположим, вы являетесь генеральным директором франшизы и рассматриваете возможность открытия новой точки в разных городах. В сети уже есть грузовики в разных городах, и у вас есть данные о прибылях и населении в городах. Моделируя прибыль как линейную функцию от населения, мы ищем прямую, наилучшим образом отражающую эту зависимость.

Начнем с импорта некоторых библиотек и изучения данных.

In [ ]:
import numpy as np
import pandas as pd

%matplotlib notebook
import matplotlib.pyplot as plt

Читаем данные, там два столбика. Посмотрим на размер, статистические характеристики, и нарисуем данные

In [ ]:
import os
path = '../data/ex1data1.txt'
data = pd.read_csv(path, header=None, names=['Population', 'Profit'])
data.head()

In [ ]:
data.describe()

In [ ]:
fig = plt.figure(figsize=(9,6))
plt.scatter(data['Population'], data['Profit'])
plt.plot([0, data['Population'].max()*1.1], [data['Profit'].mean(), data['Profit'].max()*0.9], 'r--')
plt.annotate("модель", (0,data['Profit'].mean()-1), color="red")
plt.xlim(left=0)
plt.show()

### Функция потерь

Наша модель — прямая задается двумя параметрами $\theta = (\theta_0, \theta_1)$, 
и предсказывает прибыль так:
$$\hat{y}(x) = \theta_0 + \theta_1 x$$

Ошибка модели для заданного $x_i$ (населения) есть разность между предсказанной и реальной прибылью: $\hat{y}(x_i)-y_i = \theta_0 + \theta_1 x_i - y_i$. Определим *функцию потерь* как половину среднего квадрата ошибки:
$$ MSE(\theta) = \displaystyle\frac{1}{2n}\sum^{n}_{i=1}\left(\hat{y}(x_i)-y_i\right)^2 = \displaystyle\frac{1}{2n}\sum^{n}_{i=1}\left( \theta_0 + \theta_1 x_i - y_i \right)^2.$$

Если данных много, то функция, определенная как питоновский цикл от 1 до n, будет работать медленно. Надо использовать векторные операции NumPy. Будем прицеливаться на общий случай, когда прибыль зависит от m параметров $x^{1}, x^{2}, \ldots, x^{m}$, модель выглядит как $\hat{y} = \theta_0 + \theta_1 x^1 + \theta_2 x^2 +\ldots + \theta_m x^m$, поэтому вычисление $\hat{y}$ тоже надо делать векторно, как $\theta \cdot x$. Но надо как-то включить $\theta_0$.

$$\hat{y} = \theta_0\cdot 1 + \theta_1 \cdot x^1 + \theta_2 \cdot x^2 + \ldots +\theta_m \cdot x^m$$

Мы сделаем это, добавляя фиктивный столбик единиц в data:

In [ ]:
data.insert(0, 'Ones', np.ones(len(data)))
data

Выделим все столбики, кроме $y$, в матрицу, и умножим ее на вектор параметров $\theta$

$$\hat{y} = X \left[ \begin{array}{c}\theta_0 \\ \theta_1\end{array} \right] =
\begin{array}{|cc|}
 \hline
 1 & 6.1\cr
 1 & 5.5\cr
 ...\cr
 1 & 5.4\cr
 \hline
\end{array}
\cdot
\begin{array}{|c|}
 \hline
 \theta_0 \cr
 \theta_1 \cr
 \hline
\end{array} = 
\begin{array}{|l|}
\hline
 1\cdot \theta_0 + x_1 \cdot \theta_1\cr
 1\cdot \theta_0 + x_2 \cdot \theta_1\cr
 ...\cr
 1\cdot \theta_0 + x_{n}\cdot \theta_1\cr
 \hline
\end{array}
$$
Получим матрицу ошибок. Остается вычесть столбик из $y_i$ (`Profit` в датафрейме), возвести полученную разность поэлементно в квадрат, и вычислить среднее
$$MSE(\theta) = \displaystyle\frac{1}{2n}\sum^{n}_{i=1}\left( \theta_0 + \theta_1 x_i - y_i \right)^2.$$

In [ ]:
# Создадим X (обучающую выборку) и y (целевой вектор)
X = data.iloc[:,:-1].to_numpy()
y = data['Profit'].to_numpy().reshape((-1,1))

def MSE(theta):
    return np.mean((X.dot(np.array(theta).reshape(2,1)) - y)**2) / 2

In [ ]:
# проверим совпадение вычисления в питоновском цикле с векторным numpy
theta = [1, 2]
pythonSE = 0
for i in range(len(data)):
    pythonSE +=(theta[0] + theta[1]*data['Population'][i] - data['Profit'][i])**2
pythonMSE = pythonSE/(2*len(data))
pythonMSE, MSE(theta)

### Градиентный спуск — случай одной переменной c одним параметром

В качестве разминки рассмотрим упрощенную задачу с одним параметром. Для этого будем искать наилучшее приближение только среди прямых, проходящие через начало координат, то есть в качестве вектора параметров $\theta$ брать только векторы с $\theta_0=0$. В этом случае формула для функции потерь превращается в 
$$MSE_1(\theta_1) = \displaystyle\frac{1}{2n}\sum^{n}_{i=1}\left( \theta_1 x_i - y_i \right)^2.$$
Нарисуем, как меняется эта функция при изменении нашего единственного параметра $\theta \in (0, 5)$.

In [ ]:
def MSE1(th1):
    return MSE([0, th1])

In [ ]:
from matplotlib import animation

MAX = 5


def setupAxes(left, right):
    # настройка отображения осей
    left.set_xlim(-2, MAX)
    left.set_ylim(0, mse[-1])
    left.set_xlabel(r'$\theta_1$')
    left.set_ylabel(r'$MSE(\theta_1)$')
    left.grid()
    
    right.set_xlim(x[0], x[-1])
    right.set_ylim(-5, 1.1 * data['Profit'].max())
    right.set_xlabel('x')
    right.set_ylabel('y')
    right.grid()

fig, (left, right) = plt.subplots(1,2, figsize=(9,5))

# левый график - зависимость функции потерь от выбора параметра
theta1 = np.linspace(-2, MAX, 300)                    # диапазон значений параметра theta1
mse = [MSE1(th1) for th1 in theta1]                   # значения функции потерь при этих параметрах 
plot_thetaMSE, = left.plot(theta1, mse)               # график MSE(theta1)
cur_loss = left.scatter(theta1[0], mse[0], color='r')


# правый график - данные и текущее приближение модели
x = np.linspace(0, 1.1*data['Population'].max(), 300) # диапазон значения населения
scatter_data = right.scatter(data['Population'], data['Profit']) #  точки обучающей выборки
cur_model, = right.plot(x, [0]*len(x), 'r--')

setupAxes(left, right)

def animate(i):
    cur_loss.set_offsets((theta1[i], mse[i])) # рисуем текущую точку "параметр - потери"
    cur_model.set_data(x, theta1[i]*x)        # рисуем график модели, соответствующей параметру
    
ani = animation.FuncAnimation(fig, animate, frames=len(theta1), interval=10, repeat=True)


Будем искать оптимум итерационно, получая последовательность $\theta_1^i$. Заметим, что градиент для функции одной переменной - это просто производная. Рассмотрим правило получения последовательных приближений в виде
$$\theta_1^{i+1} = \theta_1^i - \alpha MSE_1'(\theta_1^i), $$
где $\alpha$ — некоторое положительное число, называемое скоростью обучения. Заметим, что если функция возрастает при движении вправо, то ее производная положительна, а значит, из-за добавки $-\alpha MSE'$ следующее значение параметра будет левее текущего.

Наоборот, если при текущем значении параметра $\theta^i_1$ функция убывает при движении вправо, то ее производная отрицательна, а значит, из-за добавки $-\alpha MSE'$ следующее значение параметра будет правее текущего.

Таким образом, при небольших значениях $\alpha$ мы получим движение в правильном направлении — в сторону минимума функции потерь.

Найдем производную функции потерь по $\theta_1$:
$$MSE_1'(\theta_1) = \left(\displaystyle\frac{1}{2n}\sum^{n}_{i=1}\left( \theta_1 x_i - y_i \right)^2\right)' = \displaystyle\frac{1}{2n}\sum^{n}_{i=1}\left(\left( \theta_1 x_i - y_i \right)^2\right)' = \displaystyle\frac{1}{2n}\sum^{n}_{i=1}2(\theta_1 x_i - y_i)\cdot\left( \theta_1 x_i - y_i \right)' = \displaystyle\frac{1}{n}\sum^{n}_{i=1}(\theta_1 x_i - y_i)x_i$$
То есть производная половины среднеквадратичной ошибки — это среднее арифметическое ошибок, умноженных на значения признаков.

Реализуем это в виде функции и построим итерационно до 20 точек (либо до нахождения параметра, дающего производную меньше $10^{-4}$) по формуле.

In [ ]:
# Создадим X (обучающую выборку) и y (целевой вектор)
cols = data.shape[1]
X = data.iloc[:,0:cols-1].to_numpy()
y = data.iloc[:,cols-1:cols].to_numpy().reshape((-1,1))

def MSE(X, y, theta):
    return np.mean((X.dot(theta.reshape(2,1)) - y)**2) / 2

In [ ]:
ALPHA = 0.05
TH1 = [4.5]
th1_old = TH1[-1]
dmse = dMSE1(th1_old, data)
while abs(dmse)>=0.0001 and len(TH1)<20:
    dmse = dMSE1(th1_old, data)
    th1_new = th1_old - ALPHA * dmse
    TH1.append(th1_new)
    th1_old = th1_new
TH1, dmse

Выполните клетку ниже ▼.

Как видно, процесс расходится. Это вызвано тем, что значения $\theta_1$ и $MSE_1(\theta_1)$ различаются на два порядка. Поэтому производная очень велика и при выбранной скорости обучения $\alpha$, на следующей итерации мы не просто не сдвигаемся ближе к минимуму, а "перелетаем" через него в точку, которая еще дальше от него, чем текущая. 

**ЗАДАНИЯ:**  

1. Подберите скорость обучения `ALPHA` в клетке выше ▲ так, чтобы появилась сходимость; 
2. добавьте еще одну клетку, где покажите значение найденного оптимального параметра;
3. перезапустите анимацию в клетке ниже ▼. Сохраните тетрадку.

In [ ]:
from matplotlib import animation

MAX = 5

fig, (left, right) = plt.subplots(1,2, figsize=(9,5))

# левый график - зависимость функции потерь от выбора параметра и приближения
theta1 = np.linspace(-2, MAX, 300)                        # диапазон значений параметра
mse = [MSE1(th1, data) for th1 in theta1]                # значения функции потерь при этих параметрах 
left.set_xlim(-2, MAX)
left.set_ylim(0, mse[-1])
left.set_xlabel(r'$\theta_1$')
left.set_ylabel(r'$MSE(\theta_1)$')
plot_thetaMSE, = left.plot(theta1, mse)

mse1s = [MSE1(th1, data) for th1 in TH1]
points = np.array(list(zip(TH1, mse1s)))
cur_loss = left.scatter(*points[0], color='r', s=10)

# правый график - данные и текущее приближение модели
x = np.linspace(0, 1.1*data['Population'].max(), 300) # значения населения, на которых рисуем второй график
right.set_xlim(x[0], x[-1])
right.set_ylim(-5, 1.1*data['Profit'].max())
right.set_xlabel('x')
right.set_ylabel('y')
right.grid()
scatter_data = right.scatter(data['Population'], data['Profit'])
cur_model, = right.plot(x, [0]*len(x), 'r--')

def animate(i):
    cur_loss.set_offsets(points[:i]) # рисуем приближения "параметр_i - потери_i"
    cur_model.set_data(x, TH1[i]*x)        # рисуем график модели, соответствующей параметру
    
ani = animation.FuncAnimation(fig, animate, frames=len(TH1), interval=500, repeat=True)


### Градиентный спуск — случай одной переменной c двумя параметрами

Вернемся к случаю, когда прямая может сдвигаться от начала координат, то есть в приближении 

$$\hat{y}(x) = \theta_0 + \theta_1x$$

параметр $\theta_0$ не равен 0. Теперь функция потерь зависит от двух переменных и вместо производной у нее будет градиент. 

$$MSE(\theta) = \displaystyle\frac{1}{2n}\sum^{n}_{i=1}\left( \theta_0 + \theta_1 x_i - y_i \right)^2.$$

$$\mathop{\mathrm{grad}} MSE = 
\left[
\begin{array}{c} 
\displaystyle\frac{\partial MSE(\theta)}{\partial \theta_0} \\ 
\displaystyle\frac{\partial MSE(\theta)}{\partial \theta_1} 
\end{array}
\right] = 
\left[
\begin{array}{c} 
\displaystyle\frac{1}{n}\sum^{n}_{i=1}\left( \theta_0 + \theta_1 x_i - y_i \right) \\ 
\displaystyle\frac{1}{n}\sum^{n}_{i=1}\left( \theta_0 + \theta_1 x_i - y_i \right)x_i 
\end{array}
\right]
$$

Давайте нарисуем ее в виде контурного графика и нанесем векторы градиентов в некоторых точках

In [ ]:
N, M = 401, 351
th0 = np.linspace(-40, 40, N)
th1 = np.linspace(-2, 5, M)
TH0, TH1 = np.meshgrid(th0, th1)

th0m = TH0.reshape(M, N, 1)
th1m = TH1.reshape(M, N, 1)
Xm = data['Population'].to_numpy().reshape(1,1,-1)
ym = data['Profit'].to_numpy().reshape(1,1,-1)

zMSE = ((th0m + th1m*Xm - ym)**2).mean(axis=2) / 2

t0 = np.linspace(-40, 40, 9)
t1 = np.linspace(-2, 5, 8)
T0, T1 = np.meshgrid(t0, t1)
t0m = T0.reshape(8, 9, 1)
t1m = T1.reshape(8, 9, 1)
dMSEdtg0 =  (t0m + t1m*Xm - ym).mean(axis=2) 
dMSEdtg1 = ((t0m + t1m*Xm - ym)*Xm).mean(axis=2) 

G = np.gradient(zMSE)
dMSEdt0 = G[0][::50,::50]
dMSEdt1 = G[1][::50,::50]


fig, ax = plt.subplots()
ax.set_xlabel(r'$\theta_0$')
ax.set_ylabel(r'$\theta_1$')
ax.set_title(r'$MSE(\theta)$')
ax.grid()
ax.contour(TH0,TH1,zMSE, levels=50)
ax.quiver(TH0[::50,::50], TH1[::50,::50], dMSEdt0, dMSEdt1)
ax.quiver(T0, T1, dMSEdtg0, dMSEdtg1, color='red')

plt.show()

In [ ]:
th0.shape, th1.shape

In [ ]:
th0[200], th1[150]

In [ ]:
((1.5*data['Population']-data['Profit'])**2).mean()/2

In [ ]:
TH0[150,200]

In [ ]:
TH0.shape

In [ ]:
zMSE[150,200]

In [ ]:
T0.shape

В этом случае итерационная формула вместо 

$$\theta_1^{i+1} = \theta_1^i - \alpha MSE_1'(\theta_1^i) $$

принимает вид

$$\left[\begin{array}{c} \theta_0^{i+1} \\ \theta_1^{i+1}\end{array}\right]= \left[\begin{array}{c} \theta_0^{i} \\ \theta_1^{i}\end{array}\right] - \alpha \mathop{\mathrm{grad}} MSE(\theta^i) = 
\left[\begin{array}{c} \theta_0^{i} \\ \theta_1^{i}\end{array}\right] - \alpha \left[\begin{array}{c} \frac{\partial MSE(\theta^{i})}{\partial \theta_0} \\ \frac{\partial MSE(\theta^{i})}{\partial \theta_0} \end{array}\right] $$

Now let's implement linear regression using gradient descent to minimize the cost function.  The equations implemented in the following code samples are detailed in "ex1.pdf" in the "exercises" folder.

First we'll create a function to compute the cost of a given solution (characterized by the parameters theta).

The cost function is expecting numpy matrices so we need to convert X and y before we can use them.  We also need to initialize theta.

In [ ]:
X = np.matrix(X.values)
y = np.matrix(y.values)
theta = np.matrix(np.array([0,0]))

Here's what theta looks like.

In [ ]:
theta

Let's take a quick look at the shape of our matrices.

In [ ]:
X.shape, theta.shape, y.shape

Now let's compute the cost for our initial solution (0 values for theta).

In [ ]:
computeCost(X, y, theta)

So far so good.  Now we need to define a function to perform gradient descent on the parameters theta using the update rules defined in the text.

In [ ]:
def gradientDescent(X, y, theta, alpha, iters):
    temp = np.matrix(np.zeros(theta.shape))
    parameters = int(theta.ravel().shape[1])
    cost = np.zeros(iters)
    
    for i in range(iters):
        error = (X * theta.T) - y
        
        for j in range(parameters):
            term = np.multiply(error, X[:,j])
            temp[0,j] = theta[0,j] - ((alpha / len(X)) * np.sum(term))
            
        theta = temp
        cost[i] = computeCost(X, y, theta)
        
    return theta, cost

Initialize some additional variables - the learning rate alpha, and the number of iterations to perform.

In [ ]:
alpha = 0.01
iters = 1000

Now let's run the gradient descent algorithm to fit our parameters theta to the training set.

In [ ]:
g, cost = gradientDescent(X, y, theta, alpha, iters)
g

Finally we can compute the cost (error) of the trained model using our fitted parameters.

In [ ]:
computeCost(X, y, g)

Now let's plot the linear model along with the data to visually see how well it fits.

In [ ]:
x = np.linspace(data.Population.min(), data.Population.max(), 100)
f = g[0, 0] + (g[0, 1] * x)

fig, ax = plt.subplots(figsize=(12,8))
ax.plot(x, f, 'r', label='Prediction')
ax.scatter(data.Population, data.Profit, label='Traning Data')
ax.legend(loc=2)
ax.set_xlabel('Population')
ax.set_ylabel('Profit')
ax.set_title('Predicted Profit vs. Population Size')

Looks pretty good!  Since the gradient decent function also outputs a vector with the cost at each training iteration, we can plot that as well.  Notice that the cost always decreases - this is an example of a convex optimization problem.

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
ax.plot(np.arange(iters), cost, 'r')
ax.set_xlabel('Iterations')
ax.set_ylabel('Cost')
ax.set_title('Error vs. Training Epoch')

## Linear regression with multiple variables

Exercise 1 also included a housing price data set with 2 variables (size of the house in square feet and number of bedrooms) and a target (price of the house).  Let's use the techniques we already applied to analyze that data set as well.

In [ ]:
path = os.getcwd() + '\data\ex1data2.txt'
data2 = pd.read_csv(path, header=None, names=['Size', 'Bedrooms', 'Price'])
data2.head()

For this task we add another pre-processing step - normalizing the features.  This is very easy with pandas.

In [ ]:
data2 = (data2 - data2.mean()) / data2.std()
data2.head()

Now let's repeat our pre-processing steps from part 1 and run the linear regression procedure on the new data set.

In [ ]:
# add ones column
data2.insert(0, 'Ones', 1)

# set X (training data) and y (target variable)
cols = data2.shape[1]
X2 = data2.iloc[:,0:cols-1]
y2 = data2.iloc[:,cols-1:cols]

# convert to matrices and initialize theta
X2 = np.matrix(X2.values)
y2 = np.matrix(y2.values)
theta2 = np.matrix(np.array([0,0,0]))

# perform linear regression on the data set
g2, cost2 = gradientDescent(X2, y2, theta2, alpha, iters)

# get the cost (error) of the model
computeCost(X2, y2, g2)

We can take a quick look at the training progess for this one as well.

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
ax.plot(np.arange(iters), cost2, 'r')
ax.set_xlabel('Iterations')
ax.set_ylabel('Cost')
ax.set_title('Error vs. Training Epoch')

Instead of implementing these algorithms from scratch, we could also use scikit-learn's linear regression function.  Let's apply scikit-learn's linear regressio algorithm to the data from part 1 and see what it comes up with.

In [ ]:
from sklearn import linear_model
model = linear_model.LinearRegression()
model.fit(X, y)

Here's what the scikit-learn model's predictions look like.

In [ ]:
x = np.array(X[:, 1].A1)
f = model.predict(X).flatten()

fig, ax = plt.subplots(figsize=(12,8))
ax.plot(x, f, 'r', label='Prediction')
ax.scatter(data.Population, data.Profit, label='Traning Data')
ax.legend(loc=2)
ax.set_xlabel('Population')
ax.set_ylabel('Profit')
ax.set_title('Predicted Profit vs. Population Size')

That's it!  Thanks for reading.  In Exercise 2 we'll take a look at logistic regression for classification problems.